# Aprendendo a usar o Kaggle

Olá, seja bem vindo ao Workshop de classificadores de ML do Iris Data Science. Nós preparamos esse notebook para que você possa aprender do 0 até o conhecimento básico de uso de ML. Esse material é baseado em dados de competições e nas aulas do Prof. Pascal Yim (E.C. Lille). Os slides da apresentação estão disponíveis [aqui](https://docs.google.com/presentation/d/1_tE7RopalM4mJ96sh-ZBx7KKVsOyTRndLNh5KJrTjL0/edit?usp=sharing).

Se você nunca utilizou o Kaggle, crie sua conta e clique em "Copy and Edit" neste notebook e começe a programar!

Se você é iniciante, comece aqui pelo começo e siga as instruções. Se você já possui experiência, pode seguir a frente e tentar os desafios!

In [ ]:
import matplotlib.pyplot as matplotlib
import seaborn
import pandas
import numpy
%matplotlib inline

# Função para fazer reshape de listas 1D
def reshape(list1D):
     return numpy.array(list1D).reshape(-1,1)
    
# Função para imprimir nosso modelo de Regressão Logistica
def plot_ours(model):
    x = numpy.linspace(0,1,50)
    y = model.predict(reshape(x))
    matplotlib.figure(figsize=(4,4))
    matplotlib.plot(x, y, color="red")
    matplotlib.suptitle('Our Logistic model')
    matplotlib.xlabel('x')
    matplotlib.ylabel('y')
    matplotlib.show()
    
# Função para imprimir o modelo "padrão" de Regressão Logistica 
def plot_lr():
    logistical = lambda x: numpy.exp(x)/(1+numpy.exp(x))   
    x = numpy.linspace(-10,10,50)
    y = logistical(x)
    matplotlib.figure(figsize=(4,4))
    matplotlib.plot(x, y, color="red")
    matplotlib.suptitle('Logisitc Regression model')
    matplotlib.xlabel('x')
    matplotlib.ylabel('y')
    matplotlib.show()

plot_lr()
# Apresentar o que é um notebook a = 1, ...

Ok, neste começo iremos apresentar o que é um modelo, o que é x, y, labels/test, fit e predict. Lembre-se que toda Inteligência Artificial é um modelo matemático para y = f(x), onde x serão os dados de entrada, y os dados de resposta, e f a nossa função. No caso estamos estudando uma função da forma ```f(x) = exp(x)/(1+numpy.exp(x))```. Nós não iremos entrar na matemática por trás disso mas basta entender que ele é uma curva.

O que faremos a seguir é um "arredondador", ou um classificador de 0's e 1's. Queremos que ele faça 0.00231 -> 0 e 0.7987 -> 1, e assim em diante.

In [ ]:
from sklearn.linear_model import LogisticRegression

x = [0.0, 0.01, 0.1, 0.123, 0.12345, 0.234, 0.432, 0.6535, 0.6457457, 0.7, 0.71, 0.07, 0.8, 0.9, 1]
y = [0  , 0   , 0  , 0    , 0      , 0    , 0    , 1     , 1        , 1  , 1   , 0   , 1  , 1  , 1]

model = LogisticRegression()
model.fit(reshape(x),y)

Agora que sabemos como fazer o _fit_ do nosso modelo, vamos ver como ele se parece graficamente. E vamos colocar alguns números de entrada para ele tentar adivinhar!

In [ ]:
plot_ours(model)

test = [0.001431256, 0.136882, 0.6345345345, 0.81234791874]

result = model.predict(reshape(test))
print(result)

Ok, só que você nunca encontrará na sua vida os dados dessa forma. Eles normalmente estão armazenados em datasets que podem ser acessados por dataframes. A forma mais comum de acessar um dataset é utilizando a biblioteca pandas. Vamos ver esse dataset de dataset de \[0,1\]  que possui mais 10.000 números. (Atenção, por volta de 10% das respostas estão erradas!)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
import random

# Uma função para criar um dataset com 10.000 números entre 0 e 1 classificados como 0 ou 1 (as vezes errado)
def create_dataset():
    x = [random.random() for i in range(10000)]
    classify = lambda i: int(i > 0.5) if random.random() > 0.1 else int(not i > 0.5)
    dataset = pandas.DataFrame(x,columns=['x'])
    dataset['y'] = dataset['x'].apply(classify)
    return dataset 
    
dataset = create_dataset()
dataset

Veja só como os dados se parecem:

In [ ]:
seaborn.scatterplot(data=dataset,x='x',y='y', alpha=0.01)

Vamos aplicar novamente o que já sabemos.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

x = dataset['x']
y = dataset['y']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
model = LogisticRegression()
model.fit(reshape(x_train),y_train)

result = model.predict(reshape(x_test))
print(accuracy_score(result, y_test))
print(confusion_matrix(result, y_test))

# Mexendo com dados reais (Cancer de Mama)




Vamos predizer cancer! O dataset a seguir é sobre cancêr de mama no estado de Wisconsin. Na coluna do 'diagnosis' podemos ver os dois diagnósticos para cancer de mama: Maligno e Benigno. Vamos tentar adivinhar o diagnóstico baseado apenas nas informações médicas que temos.

In [ ]:
import tree
# Aqui mapeia os dois tipos de diagnósticos no espaço
def plot_cancer_sizes(df):
    matplotlib.figure(figsize=(12,12))
    seaborn.kdeplot(df[df['diagnosis']=='M'].perimeter_worst, df[df['diagnosis']=='M'].area_worst, cmap="Reds",  shade=True, alpha=0.3, shade_lowest=False)
    seaborn.kdeplot(df[df['diagnosis']=='B'].perimeter_worst, df[df['diagnosis']=='B'].area_worst, cmap="Greens", shade=True, alpha=0.3, shade_lowest=False)
    matplotlib.show()

# Faz um plot do perimeter_worst para os dois diagnósticos
def plot_cancer_perimeter(df):
    fig = seaborn.FacetGrid(df, hue="diagnosis", aspect=3)
    fig.map(seaborn.kdeplot, "perimeter_worst", shade=True)
    fig.add_legend()
    matplotlib.show()

# Faz um plot da árvore de decisões
def plot_tree(model,x_train):
    matplotlib.figure(figsize=(15,15))
    tree.plot_tree(model, feature_names=x_train.columns, class_names=['benigno','maligno'], fontsize=14, filled=True)
    matplotlib.show()

# Faz um plot das importâncias para um Random Forest
def plot_importances(model,df):
    importances = model.feature_importances_
    indices = numpy.argsort(importances)
    matplotlib.figure(figsize=(12,8))
    matplotlib.barh(range(len(indices)), importances[indices], color='b', align='center')
    matplotlib.yticks(range(len(indices)), df.columns[indices])
    matplotlib.suptitle('Importância das características')
    matplotlib.show()

cancer = pandas.read_csv('../input/breast-cancer-wisconsin-data/data.csv')
cancer

Primeiro passo: Tire a coluna inútil no final!

In [ ]:
cancer = cancer.drop(['Unnamed: 32'], axis=1)

Agora, vamos visualizar como o tamanho do perímetro e tamanho da área se comportam para o tipo maligno e benigno

In [ ]:
plot_cancer_sizes(cancer)

Ok, faça x com todas as colunas menos o diagnóstico, e y como diagnóstico! Lembre de separar os dados e aplicar a Regressão Logística

In [ ]:
x = cancer.drop(['diagnosis'],axis=1)
y = cancer['diagnosis']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

model = LogisticRegression()
model.fit(x_train,y_train)

result = model.predict(x_test)
print(accuracy_score(result, y_test))
print(confusion_matrix(result, y_test))

A acurácia foi ruim, como você pode ver. O que aconteceu? Será que tem alguma coluna que está fazendo nossos dados tendenciosos? (dica: tem sim)

In [ ]:
cancer = cancer.drop(['id'], axis=1)
x = cancer.drop(['diagnosis'],axis=1)
y = cancer['diagnosis']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

model = LogisticRegression()
model.fit(x_train,y_train)

result = model.predict(x_test)
print(accuracy_score(result, y_test))
print(confusion_matrix(result, y_test))

Lembre-se sempre da aplicação de IA: classificar coisas! Imagine que eu sou um médico com um paciente com essas condições. O câncer é maligno ou benigno?

In [ ]:
nosso_cancer = list({
 'radius_mean': 17.99,
 'texture_mean': 10.38,
 'perimeter_mean': 122.8,
 'area_mean': 1001.0,
 'smoothness_mean': 0.1184,
 'compactness_mean': 0.2776,
 'concavity_mean': 0.3001,
 'concave points_mean': 0.1471,
 'symmetry_mean': 0.2419,
 'fractal_dimension_mean': 0.07871,
 'radius_se': 1.095,
 'texture_se': 0.9053,
 'perimeter_se': 8.589,
 'area_se': 153.4,
 'smoothness_se': 0.006399,
 'compactness_se': 0.04904,
 'concavity_se': 0.05372999999999999,
 'concave points_se': 0.01587,
 'symmetry_se': 0.03003,
 'fractal_dimension_se': 0.006193,
 'radius_worst': 25.38,
 'texture_worst': 17.33,
 'perimeter_worst': 184.6,
 'area_worst': 2019.0,
 'smoothness_worst': 0.1622,
 'compactness_worst': 0.6656,
 'concavity_worst': 0.7119,
 'concave points_worst': 0.2654,
 'symmetry_worst': 0.4601,
 'fractal_dimension_worst': 0.1189
}.values())

model.predict(reshape(nosso_cancer).transpose())

Ok, cansamos de usar Logistic Regression. Queremos modelos diferentes para fazer aprendizado de máquina! Vamos tentar utilizar uma árvore de decisões, é o mesmo "modelo" que o Akinator funcionava!

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

model = DecisionTreeClassifier(max_depth = 3, min_samples_leaf = 20)
model.fit(x_train,y_train)

result = model.predict(x_test)
print(accuracy_score(result, y_test))
print(confusion_matrix(result, y_test))
plot_tree(model,x_train)

Ok, agora vamos ver outro tipo de modelo chamado RandomForestClassifier. Veja que o modelo de aplicação no código é o mesmo!

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train,y_train)

result = model.predict(x_test)
print(accuracy_score(result, y_test))
print(confusion_matrix(result, y_test))
plot_importances(model,cancer)

E agora vamos aprender um pouco sobre classificação, como precisão, recall, f1-score, e **LOSS**

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, result))

# IRIS

Iris, o desafio que deu nome ao nome do nosso grupo, é um dataset do tipo de flor Iris (e não o olho!). Nele, há três tipos de espécies, com as informações sobre sépalas e pétalas.

In [ ]:
def plot_iris(df):
    seaborn.pairplot(df, hue="Species")
    matplotlib.show()

iris = pandas.read_csv('../input/iris/Iris.csv')
iris

Vamos ver como cada informação se comporta para cada espécie.

In [ ]:
plot_iris(iris)

Vamos aplicar nossa regressão logística.

In [ ]:
x = iris.drop(['Species'],axis=1)
y = iris['Species']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

model = LogisticRegression()
model.fit(x_train,y_train)

result = model.predict(x_test)
print(accuracy_score(result, y_test))
print(confusion_matrix(result, y_test))

# Desafio 1: Titanic

O seu objetivo é descobrir, dado as informações de um passageiro no navio Titanic, se ele sobreviveu ou não o acidente (coluna survived). Neste caso, você perceberá que algumas colunas podem deixar seu aprendizado pior. Outro ponto importante é que há varias informações faltantes (NaN).

**Objetivos**
- Você consegue preencher as informações faltantes de alguma forma? Como?
- Tente ultrapassar 80% de precisão.




In [ ]:
titanic_train = pandas.read_csv("../input/titanic/train.csv")
titanic_test = pandas.read_csv("../input/titanic/test.csv")
titanic_train

# Desafio 2: IMDB

Ok, o seu desafio agora é utilizar Machine Learning para fazer uma Análise de Sentimentos nas reviews de filmes do IMDB. Neste dataset você possui 50.000 reviews de filmes classificadas como "positiva" e "negativa". Neste dataset, seu modelo pode demorar bastante (coisa de 5 minutos para cima). O quê você precisa fazer:

- Limpar as reviews (use a função clear_sentence para cada string do dataset)
- Separe o treinamento e teste.
- Vetorizar as palavras (pode usar o HashingVectorizer()). Procure no google como aplicar isso.
- Coloque em um Machine Learning.

**Objetivos**
- Qual é o melhor tipo de modelo de ML para esse NLP? (Dica: pense em modelos que trabalham com vetores)
- Ultrapasse 90% de precisão neste dataset demorando menos de 1 minuto para rodar (utilize time.time() para pegar os tempos.




In [ ]:
import string
import time

from sklearn.feature_extraction.text import HashingVectorizer

def clear_sentence(sentence):
    sentence = sentence.replace('<br />', ' ')
    sentence = sentence.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    sentence = sentence.lower()
    return sentence

imdb = pandas.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
imdb

# Desafio 3: Diabetes
O dataset a seguir possui dados sobre a incidência de diabetes na população do povo Pima. Seu desafio é descobrir a coluna outcome baseado nos outros dados. Seu desafio é ultrapassar 82% de precisão.

In [ ]:
diabetes = pandas.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")
diabetes

# Desafio 4: MNIST - Digit Recognizer
Dataset com dígitos escritos à mão e seus respectivos valores. Cada linha dos datasets (tanto de treino quanto de teste) está estruturada da seguinte forma:

| Digito representado | pixel 1x1 | ... | pixel 28x28 |
|:-----------------:|:---------:|:---:|:----------:|
|5|0|...|0|

Como temos uma imagem 28x28 temos 784 valores de pixel por coluna, todos valores binários:

In [ ]:
def plt_digit_from_row(row):
    label, image = mnist_train.values[row,0], mnist_train.values[row,1:]
    matplotlib.imshow(image.reshape(28,28), cmap='hot')
    matplotlib.title("Label: %s"%label)
    matplotlib.show()

mnist_train = pandas.read_csv("../input/mnist-in-csv/mnist_train.csv")
mnist_test = pandas.read_csv("../input/mnist-in-csv/mnist_test.csv")
mnist_train.head()

Conseguimos ver como é a imagem redimensionando o a matriz `1x784` para uma `28x28`:

In [ ]:
plt_digit_from_row(0)

In [ ]:
mnist_train_labels, mnist_train_values = mnist_train.values[:,0], mnist_train.values[:,1:]
mnist_test_labels, mnist_test_values = mnist_test.values[:,0], mnist_test.values[:,1:]

Você vai precisar fazer o escalamento das imagens para poder 

In [ ]:
model = LogisticRegression()

model.fit(mnist_train_values, mnist_train_labels)


prediction = model.predict(mnist_test_values)

print(classification_report(prediction, mnist_test_labels))

# Desafio 5: Fashion MNIST
Dataset com desenhos de tipos de roupa classificadas com labels
Cada linha dos datasets (tanto de treino quanto de teste) está estruturada da seguinte forma:


| Label de cada roupa | pixel 1x1 | ... | pixel 28x28 |
|:-----------------:|:---------:|:---:|:----------:|
|5|0|...|0|

Como temos uma imagem 28x28 temos 784 valores de pixel por coluna, todos valores binários:

In [ ]:
def plt_clothes_from_row(row):
    label, image = fashion_mnist_train.values[row,0], fashion_mnist_train.values[row,1:]
    matplotlib.imshow(image.reshape(28,28), cmap='gray')
    matplotlib.title("Label: %s"%label)
    matplotlib.show()
    
fashion_mnist_train, fashion_mnist_test = pandas.read_csv("../input/fashionmnist/fashion-mnist_train.csv"), pandas.read_csv("../input/fashionmnist/fashion-mnist_test.csv")
fashion_mnist_train.head()

In [ ]:
plt_clothes_from_row(0)

In [ ]:
fashion_mnist_train_labels, fashion_mnist_train_values = fashion_mnist_train.values[:,0], fashion_mnist_train.values[:,1:]
fashion_mnist_test_labels, fashion_mnist_test_values = fashion_mnist_test.values[:,0], fashion_mnist_test.values[:,1:]

In [ ]:
model = LogisticRegression()

model.fit(fashion_mnist_train_values, fashion_mnist_train_labels)

prediction = model.predict(fashion_mnist_test_values)

print(classification_report(prediction, mnist_test_labels))

# Desafio Final: Predizer eleições com Tweets

Dessa vez nem preparamos o dataset para você. Utilizando o dataset das eleições australianas, você consegue predizer que regiões da Austrália apoiam qual partido? Tente treinar seu modelo em algum dataset classificado com positivo e negativo e então faça o .fit() no dataset das eleições. Você pode utilizar a função a seguir para plotar seus dados.

In [ ]:
from mpl_toolkits.basemap import Basemap

# Precisa ter as colunas 'lat' e 'long'. Retorna o mesmo dataframe com apenas os tweets na região da austrália.
def pegar_tweets_na_australia(dataframe):
    bot_lat, top_lat, left_lon, right_lon = -44,-10,109,156
    top = dataframe.lat <= top_lat
    bot = dataframe.lat >= bot_lat
    left = dataframe.long >= left_lon
    right = dataframe.long <= right_lon
    index = top&bot&left&right 
    return dataframe[index]

# Passe seu dataframe com os dados que você quer plotar e uma legenda (como string).
def plotar_mapa(dataframe,legenda):
    Australia_map = Basemap(llcrnrlat=-44,urcrnrlat=-10,llcrnrlon=109,urcrnrlon=156)
    matplotlib.figure(figsize=(12,10))
    Australia_map.bluemarble(alpha=0.9)
    seaborn.scatterplot(x='long', y='lat', data=dataframe, alpha=1, s=200, label=legenda)
    matplotlib.show()